## 实现Bagging集成方法
使用和上节AdaBoost一样的数据集，然后自己实现Bagging算法，达到以下要求：
1.	基学习器自行选择（可以使用Scikit-learn现成的分类器），使用不同的基学习器实现2种以上的Bagging。
2.	运用Accuracy, Precision, Recall, F1四个指标进行对比评测，进行十折交叉验证，把结果绘制成表格
3.	与Scikit-learn 的BaggingClassifier得到的结果进行对比（基学习器和你自己实现的Bagging相同）
4.	详细分析不同的基学习器的Bagging的表现差异以及可能的原因。
5.	与AdaBoost的结果进行对比，分析两种集成方法的优劣


In [4]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [5]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

#### 导入数据集

In [6]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancers = load_breast_cancer()
x = cancers.data[:,:]     #获取特征值
y = cancers.target     #获取标签
print(x.shape)         #查看特征形状
print(y.shape)         #查看标签形状
print(x)
print(y,np.sum(y))

(569, 30)
(569,)
[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0

## 实现MyBagging


In [20]:
class MyBagging(object):

    def __init__(self,n_estimators,estimator,rate=1.0):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.rate = rate

    def Voting(self,data):          #投票法
        term = np.transpose(data)   #转置
        result =list()              #存储结果

        def Vote(df):               #对每一行做投票
            store = defaultdict()
            for kw in df:
                store.setdefault(kw, 0)
                store[kw] += 1
            return max(store,key=store.get)

        result= map(Vote,term)      #获取结果
        return result

    #随机欠采样函数
    def UnderSampling(self,data):
        #np.random.seed(np.random.randint(0,1000))
        data=np.array(data)
        np.random.shuffle(data)    #打乱data          
        newdata = data[0:int(data.shape[0]*self.rate),:]   #切片，取总数*rata的个数，删去（1-rate）%的样本
        return newdata   

    def TrainPredict(self,train,test):          #训练基础模型，并返回模型预测结果
        clf = self.estimator.fit(train[:,0:-1],train[:,-1])
        result = clf.predict(test[:,0:-1])
        return result

    #简单有放回采样
    def RepetitionRandomSampling(self,data,number):     #有放回采样，number为抽样的个数
        sample=[]
        for i in range(int(self.rate*number)):
             sample.append(data[random.randint(0,len(data)-1)])
        return sample

    def Metrics(self,predict_data,test):        #评价函数
        score = predict_data
        recall=recall_score(test[:,-1], score, average=None)    #召回率
        precision=precision_score(test[:,-1], score, average=None)  #查准率
        return recall,precision


    def MutModel_clf(self,train,test,sample_type = "RepetitionRandomSampling"):
        print ("self.Bagging Mul_basemodel")
        result = list()
        num_estimators =len(self.estimator)   #使用基础模型的数量

        if sample_type == "RepetitionRandomSampling":
            print ("选择的采样方法：",sample_type)
            sample_function = self.RepetitionRandomSampling
        elif sample_type == "UnderSampling":
            print ("选择的采样方法：",sample_type)
            sample_function = self.UnderSampling 
            print ("采样率",self.rate)
        elif sample_type == "IF_SubSample":
            print ("选择的采样方法：",sample_type)
            sample_function = self.IF_SubSample 
            print("采样率",(1.0-self.rate))

        for estimator in self.estimator:
            print (estimator)
            for i in range(int(self.n_estimators/num_estimators)):
                sample=np.array(sample_function(train,len(train)))       #构建数据集
                clf = estimator.fit(sample[:,0:-1],sample[:,-1])
                result.append(clf.predict(test[:,0:-1]))      #训练模型 返回每个模型的输出

        score = self.Voting(result)
        recall,precosoion = self.Metrics(score,test)
        return recall,precosoion    

## 使用Sklearn自带的BaggingClassifier

In [9]:
from sklearn.ensemble import BaggingClassifier

### 使用决策树作为基分类器

In [11]:
boost = BaggingClassifier(base_estimator = DecisionTreeClassifier())
boost.fit(x,y)
print(boost.score(x,y))
prediction = cross_val_predict(boost, x, y.astype('int'), cv = 10)

acc = accuracy_score(y,prediction)
pre = precision_score(y,prediction)
rec = recall_score(y,prediction)
f1 = f1_score(y,prediction)

print(acc,"|",pre,"|",rec,"|",f1)

0.9929701230228472
0.9507908611599297 | 0.9686609686609686 | 0.9523809523809523 | 0.9604519774011299


### 使用SVC作为基分类器

In [13]:
boost = BaggingClassifier(base_estimator=SVC(kernel='rbf',C=0.1,random_state=32,max_iter = 5000))
boost.fit(x,y)
print(boost.score(x,y))
prediction = cross_val_predict(boost, x, y.astype('int'), cv = 10)

acc = accuracy_score(y,prediction)
pre = precision_score(y,prediction)
rec = recall_score(y,prediction)
f1 = f1_score(y,prediction)

print(acc,"|",pre,"|",rec,"|",f1)

0.8927943760984183
0.8927943760984183 | 0.8627450980392157 | 0.9859943977591037 | 0.9202614379084968


### 使用LinearSVC作为基分类器

In [21]:
boost = BaggingClassifier(base_estimator=LinearSVC(C=0.1,random_state=32,max_iter = 5000))
boost.fit(x,y)
print(boost.score(x,y))
prediction = cross_val_predict(boost, x, y.astype('int'), cv = 10)

acc = accuracy_score(y,prediction)
pre = precision_score(y,prediction)
rec = recall_score(y,prediction)
f1 = f1_score(y,prediction)

print(acc,"|",pre,"|",rec,"|",f1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

0.9314586994727593


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

0.9261862917398945 | 0.9177718832891246 | 0.969187675070028 | 0.9427792915531334


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


基学习器|精度|查准率|查全率|F1
-|-|-|-|-
DecisionTree |0.9507908611599297 | 0.9686609686609686 | 0.9523809523809523 | 0.9604519774011299
SVC(rbf) |0.8927943760984183 | 0.8627450980392157 | 0.9859943977591037 | 0.9202614379084968
LinearSVC |0.9068541300527241 | 0.9108108108108108 | 0.9439775910364145 | 0.9270976616231087
